# Computational and Artifical Intelligenge Assignment 1

Problem Solving by Searching using IDA* and Best First Search

Weightage: 10%

Date of Posting - 04/July/2019

Deadline - 19/July/2019

Please enter group and individual information here

Group ID/Number:

Group Members BITS ID:

PAMIDIPATI SAI LAKSHMI THANUJA : 2018AH04106
GANNAVARAPU S S VENKATA SUBBA RAO : 2018AH04108
Thiraviam Ramaiah Pillai : 2018AH04118

Things to follow

1. Use appropriate data structures for the fringes and explain the reasoning behind the usage. You need not create data structures, instead use available libraries directly.

2. Avoid any hard-coding unless absolutely necessary.

3. Provide proper documentation

Coding begins now!!!

Define the environment representation in the following code block

In [133]:
#Environment representation goes here
import sys
import math
import queue as q
import time
import numpy as np
import heapq as hpq_BFS
from collections import deque
 
environment = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
               [1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1],
               [1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1],
               [1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1],
               [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
               [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
               [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1],
               [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
    

start = [0, 2]
end = [5, 10]
tot_rows = len(environment)
tot_columns = len(environment[0])
row_moves = [0, -1, 0, 1]
column_moves = [-1, 0, 1, 0]

q_IDAstar = q.Queue(maxsize=tot_rows*tot_columns)
q_BFS = q.Queue(maxsize=tot_rows*tot_columns)

Define the fringe (data structure) and its supported methods in the following code block

In [134]:
def heuristic_IDAstar(current_row,current_column,end):
    # Calculating Euclidean distance    
    heuristic_cost = round(math.sqrt(math.pow((current_row - end[0]),2) + math.pow((current_column - end[1]),2)),2)    
    return heuristic_cost

def heuristic_BFS(current_row,current_column,end):
    heuristic_cost = round(math.sqrt(math.pow((current_row - end[0]),2) + math.pow((current_column - end[1]),2)),2)    
    return heuristic_cost

# Node Class for IDA* Path finding 

class agent_coordinates():
    
    def __init__(self,row,column):
        self.row = row
        self.column = column

class Node():
    
    def __init__(self,parent=None,position=None):
        self.parent = parent
        self.position = position        

        self.g_cost = 0
        self.h_cost = 0
        self.f_cost = 0
        self.traveresed = 0

    def __eq__(self, other):
        return self.position == other.position

def get_agent_position(current_row,current_column,tot_rows, tot_columns):
    if (current_row >=0 and current_row < tot_rows) and (current_column >=0 and current_column < tot_columns):
        return True
    return False    

def generate_successors(current_node):
    successors = []
    coordinates = current_node.position

    for new_position in range(0,4):
        new_row = coordinates.row + row_moves[new_position]
        new_column = coordinates.column + column_moves[new_position]
        if (get_agent_position(new_row,new_column,tot_columns,tot_columns)) and environment[new_row][new_column] != 0:
            successors.append(Node(current_node,agent_coordinates(new_row,new_column)))  

    return successors

def min_f_cost(f_limit,successors):
    f_min_cost_node = []
    for child in successors: 
        location = child.position
        if (child.f_cost >= f_limit):            
            f_min_cost_node.append([child, child.f_cost])
    min_f_cost_node = sorted(f_min_cost_node, key = lambda x:x[1])    
    return min_f_cost_node[0][0]

def is_Exit_Location(row,column,destination):
    location = []
    location.append([row, column])
    if (location[0] == destination):
        return False
    elif (location[0] != destination):
        return True
    
def bfs_min(list_of_nodes,bfs_cost):
    for item in list_of_nodes:
        if((item.bfs_cost == bfs_cost) and item.traveresed !=1):
            return item

Define your IDA* in the following code block

In [135]:
#Algorithm IDA*
def ida_star(node,destination=end): 
    
    agent_path = []
    optimal_path_cost = []
    q_IDAstar.put(node)
    coordinates = node.position
    nodes_mem = []
    
    while (not q_IDAstar.empty() and (is_Exit_Location(coordinates.row,coordinates.column,destination))):        
        node = q_IDAstar.get() 
        coordinates = node.position
       
        if (get_agent_position(coordinates.row,coordinates.column,tot_rows,tot_columns)) and environment[coordinates.row][coordinates.column] != 0:
            if (node.parent is None):
                f_limit = heuristic_IDAstar(coordinates.row,coordinates.column,destination)
            
            children = generate_successors(current_node=node)            
            agent_path.append([coordinates.row, coordinates.column])

            for child in children:
                child.g_cost = node.g_cost + 1
                child_coordinates = child.position
                child.h_cost = heuristic_IDAstar(child_coordinates.row,child_coordinates.column,destination)
                child.f_cost = child.g_cost + child.h_cost
                nodes_mem.append(node)
            
            optimal_node = min_f_cost(f_limit,children)  
            new_coordinates = optimal_node.position
            f_limit = optimal_node.f_cost 
            optimal_path_cost.append(optimal_node.f_cost)
            
            if (is_Exit_Location(new_coordinates.row,new_coordinates.column,destination)):
                q_IDAstar.put(optimal_node)
            else:
                optimal_path_cost.append(optimal_node.f_cost)
                agent_path.append([new_coordinates.row, new_coordinates.column])
                memory_nodes = len(nodes_mem)

    return agent_path,optimal_path_cost,memory_nodes


Define your Greedy Best First Search algorithm in the following code block

In [136]:
def bfs(node,destination):
    
    start_list = []
    end_list = []    
    min_list = []
    bfs_agent_path = []
    opt_path_cost = []
    q_BFS.put(node)
    coordinates = node.position
    
    while (not q_BFS.empty() and (is_Exit_Location(coordinates.row,coordinates.column,destination))):
        node = q_BFS.get()
        coordinates = node.position        

        if (get_agent_position(coordinates.row,coordinates.column,tot_rows,tot_columns)) and environment[coordinates.row][coordinates.column] != 0:
            if (node.parent is None):
                node.bfs_cost = heuristic_BFS(coordinates.row,coordinates.column,destination)                
                hpq_BFS.heappush(min_list,node.bfs_cost)
                start_list.append(node)

            if (len(start_list)!=0 and (node in start_list and node.traveresed == 1)):
                start_list.remove(node)
                end_list.append(node)
            
            children = generate_successors(current_node=node)
            bfs_agent_path.append([coordinates.row,coordinates.column])

            for child in children:
                child_coordinates = child.position
                child.h_cost = heuristic_BFS(child_coordinates.row,child_coordinates.column,destination)
                child.bfs_cost = child.h_cost
                start_list.append(child)
                hpq_BFS.heappush(min_list,child.bfs_cost)

            hpq_BFS.heapify(min_list)
            bfs_cost = hpq_BFS.heappop(min_list)
            opt_path_cost.append(round(bfs_cost,2)) 
            min_cost_node = bfs_min(start_list + end_list,bfs_cost)
            new_coordinates = min_cost_node.position

            if (is_Exit_Location(new_coordinates.row,new_coordinates.column,destination)):
                    q_BFS.put(min_cost_node)
            else:
                mem_nodes_bfs = len(start_list+end_list)
                opt_path_cost.append(round(min_cost_node.bfs_cost,2))
                bfs_agent_path.append([new_coordinates.row, new_coordinates.column])           

    return bfs_agent_path,opt_path_cost,mem_nodes_bfs

Feel free to add code blocks for any other methods needed starting here.

In [ ]:
#Code Block 1


Call your main function/algorithm for IDA* in the next code block with appropriate input representation

In [137]:
#Computation call for IDA*
current_node = Node(None,agent_coordinates(start[0],start[1]))
agent_path_IDA,path_cost_IDA,memory_nodes=ida_star(current_node,end)


Call your main function/algorithm for Best First Search in the next code block with appropriate input representation

In [138]:
#Computation call for Best First Search
current_node = Node(None,agent_coordinates(start[0],start[1]))
agent_path_BFS,path_cost_BFS,mem_nodes_bfs=bfs(current_node,end)

The agent should provide expected output for questions mentioned below in the subsequent blocks

(1) Path taken to reach destination from source for IDA*

In [139]:
# Execute statement to retrieve the path taken here using IDA*
print("Path taken here using IDA* ----",agent_path_IDA)

Path taken here using IDA* ---- [[0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [1, 9], [2, 9], [3, 9], [4, 9], [4, 10], [5, 10]]


(1) Path taken to reach destination from source for Best First Search

In [140]:
# Execute statement to retrieve the path taken here using Best First Search
print("Path taken using Best First Search ----" ,agent_path_BFS)

Path taken using Best First Search ---- [[0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [1, 9], [2, 9], [3, 9], [4, 9], [4, 10], [5, 10]]


(2) Cost of the path for IDA* here

In [142]:
# Execute statement to retrieve the cost of the path here using IDA*
print("Cost of the path using IDA*---",path_cost_IDA)

Cost of the path using IDA*--- [9.6, 9.809999999999999, 10.07, 10.4, 10.83, 11.39, 12.1, 12.120000000000001, 12.16, 12.24, 12.41, 13.0, 13.0, 13.0]


In [143]:
print("Total cost through IDA* for optimal path ----", sum(path_cost_IDA))

Total cost through IDA* for optimal path ---- 162.13


(2) Cost of the path for Best First Search here

In [144]:
# Execute statement to retrieve the cost of the path here using Best First Search
print("Cost of the path using Best First Search ---" ,path_cost_BFS)

Cost of the path using Best First Search --- [8.6, 7.81, 7.07, 6.4, 5.83, 5.39, 5.1, 4.12, 3.16, 2.24, 1.41, 1.0, 0.0, 0.0]


(3) Total Number of nodes expanded to get this state using IDA*

In [145]:
# Execute statement to retrieve the total number of nodes expanded to get this state using IDA* here
print("Total nodes expanded to get this state with IDA* -----",  len(agent_path_IDA))

Total nodes expanded to get this state with IDA* ----- 14


(3) Total Number of nodes expanded to get this state using Best First Search

In [146]:
# Execute statement to retrieve the total number of nodes expanded to get this state using Best First Search here
print("Total nodes expanded to get this state with Best First Search -----",  len(agent_path_BFS))

Total nodes expanded to get this state with Best First Search ----- 14


(4) Maximum number of nodes kept at the memory at any point in time using IDA*

In [147]:
# Execute Output for the maximum number of nodes kept at the memory at any point in time using IDA* here
print("Maximum number of nodes kept at the memory at any point in time using IDA* --- " ,memory_nodes)

Maximum number of nodes kept at the memory at any point in time using IDA* ---  37


(4) Maximum number of nodes kept at the memory at any point in time using Best First Search

In [148]:
# Execute Output for the maximum number of nodes kept at the memory at any point in time using Best First Search here
print("Maximum number of nodes kept at the memory at any point in time using Best First Search----" ,mem_nodes_bfs)

Maximum number of nodes kept at the memory at any point in time using Best First Search---- 38


All the best!! Happy Coding!!
Let human intelligence prevail